In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rm -rf Wav2Vec2_PyCTCDecode
!git clone --recursive https://github.com/patrickvonplaten/Wav2Vec2_PyCTCDecode.git

Cloning into 'Wav2Vec2_PyCTCDecode'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 49 (delta 22), reused 15 (delta 5), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [3]:
!rm -rf ctcdecode
%cd /content
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .
%cd /content

/content
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 2.93 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14102, done.        
remote: Counting objects: 100% (415/415), done.        
remote: Compressing objects: 100% (290/290), done.        
remote: Total 14102 (

In [4]:
!pip install -q torchaudio
!pip install -q transformers>=4.12.0
!pip install -q datasets
!pip install -q lang_trans
!pip install -q arabic_reshaper
!pip install -q python-bidi
!pip install -q pydub
!pip install -q soundfile
!pip install -q jiwer
!pip install -q PyArabic
!pip install -q ctc-segmentation
!pip install -q pyctcdecode
!pip install -q ffmpeg-python
!pip install -q https://github.com/kpu/kenlm/archive/master.zip

     |████████████████████████████████| 365 kB 7.4 MB/s 
     |████████████████████████████████| 212 kB 67.3 MB/s 
     |████████████████████████████████| 115 kB 87.3 MB/s 
     |████████████████████████████████| 127 kB 77.9 MB/s 
     |████████████████████████████████| 1.4 MB 8.1 MB/s 
     |████████████████████████████████| 1.6 MB 56.7 MB/s 
     |████████████████████████████████| 104 kB 96.3 MB/s 
     |████████████████████████████████| 126 kB 8.7 MB/s 
     |████████████████████████████████| 73 kB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 390 kB 14.8 MB/s 
     |████████████████████████████████| 550 kB 5.1 MB/s 


In [5]:
%cd Wav2Vec2_PyCTCDecode

/content/Wav2Vec2_PyCTCDecode


In [6]:
!wget https://raw.githubusercontent.com/MagedSaeed/arabic-poetry-speech-classification/main/audio_utils.py

--2022-09-19 03:09:29--  https://raw.githubusercontent.com/MagedSaeed/arabic-poetry-speech-classification/main/audio_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3365 (3.3K) [text/plain]
Saving to: ‘audio_utils.py’

audio_utils.py      100%[===================>]   3.29K  --.-KB/s    in 0s      

2022-09-19 03:09:29 (55.9 MB/s) - ‘audio_utils.py’ saved [3365/3365]



In [7]:
import transformers
transformers.utils.move_cache()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [8]:
import os
import re
import sys
import json
import tensorflow as tf
import torch
import jiwer
import kenlm
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
from tqdm import tqdm
import soundfile as sf
import arabic_reshaper
import ctc_segmentation
from jiwer import wer,cer
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
from dataclasses import asdict
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from audio_utils import get_audio
from argparse import ArgumentParser
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from pyctcdecode import build_ctcdecoder
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from sklearn.metrics import accuracy_score
from typing import Any, Dict, List, Optional, Union
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset,load_metric
from keras.preprocessing.sequence import pad_sequences
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
%matplotlib inline

## Helper Classes and methods

In [33]:
class Transcriber:
    def __init__(self, model_path, processor_path="bakrianoo/sinai-voice-ar-stt"):
        self.processor = Wav2Vec2Processor.from_pretrained(processor_path)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_path).eval()
        self.model.to("cuda")

    def transcribe(self, file_path=None,speech_array=None, with_ctc=True):
        assert file_path or speech_array, "file path or speech_array should be passed"
        if file_path:
          speech, sampling_rate = librosa.load(file_path, sr=16_000)
        else:
          speech = np.array(speech_array,dtype=np.float32)
        inputs = self.processor(
            speech, sampling_rate=16_000, return_tensors="pt", padding=True
        )
        with torch.no_grad():
          logits = self.model(inputs.input_values.to('cuda')).logits
          for logit in logits:
            logit[logit==-100] = self.processor.tokenizer.pad_token_id
          predicted = torch.argmax(logits, dim=-1)
        # predicted[predicted == -100] = self.processor.tokenizer.pad_token_id
        predicted_tokenized = self.processor.tokenizer.batch_decode(predicted)
        clean_predicted_tokenized = [araby.strip_tashkeel(araby.strip_tatweel(predicted_tokenized[0]))]
        return predicted_tokenized

In [10]:
lm_path = "/content/drive/MyDrive/KFUPM-Master/ICS606/Models/baits_dataset_kenlm_model.arpa"

In [11]:
!python fix_lm.py --path_to_ngram /content/drive/MyDrive/KFUPM-Master/ICS606/Models/baits_dataset_kenlm_model.arpa --path_to_fixed fixed_baits_dataset_kenlm.arpa

In [12]:
fixed_lm_path = 'fixed_baits_dataset_kenlm.arpa'

In [34]:
class CTCTranscriber:
    def __init__(self, model_path, lm_model_path='fixed_baits_dataset_kenlm.arpa', processor_path="bakrianoo/sinai-voice-ar-stt"):
        self.processor = Wav2Vec2Processor.from_pretrained(processor_path)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_path).eval()
        self.model.to("cuda")
        self.lm_model_path = lm_model_path
        self.decoder = build_ctcdecoder(
            self.vocab,
            self.lm_model_path,
            alpha=0.5,
        )

    @property
    def vocab(self):
      vocab_dict = self.processor.tokenizer.get_vocab()
      sort_vocab = sorted((value, key) for (key,value) in vocab_dict.items())
      vocab = [token for _,token in sort_vocab]
      return vocab

    def transcribe(self, file_path=None,speech_array=None, with_ctc=True):
        assert file_path or speech_array, "file path or speech_array should be passed"
        if file_path:
          speech, sampling_rate = librosa.load(file_path, sr=16_000)
        else:
          speech = np.array(speech_array,dtype=np.float32)
        inputs = self.processor(
            speech, sampling_rate=16_000, return_tensors="pt", padding=True
        )
        with torch.no_grad():
          logits = self.model(inputs.input_values.to('cuda')).logits
          for logit in logits:
            logit[logit==-100] = self.processor.tokenizer.pad_token_id
          decoded = [araby.strip_tashkeel(self.decoder.decode(logits.cpu().numpy()[0]))]
        return decoded

In [35]:
char2idx = {
 ' ': 0,
 'ء': 1,
 'آ': 2,
 'أ': 3,
 'ؤ': 4,
 'إ': 5,
 'ئ': 6,
 'ا': 7,
 'ب': 8,
 'ة': 9,
 'ت': 10,
 'ث': 11,
 'ج': 12,
 'ح': 13,
 'خ': 14,
 'د': 15,
 'ذ': 16,
 'ر': 17,
 'ز': 18,
 'س': 19,
 'ش': 20,
 'ص': 21,
 'ض': 22,
 'ط': 23,
 'ظ': 24,
 'ع': 25,
 'غ': 26,
 'ف': 27,
 'ق': 28,
 'ك': 29,
 'ل': 30,
 'م': 31,
 'ن': 32,
 'ه': 33,
 'و': 34,
 'ى': 35,
 'ي': 36,
 'ً': 37,
 'ٌ': 38,
 'ٍ': 39,
 'َ': 40,
 'ُ': 41,
 'ِ': 42,
 'ّ': 43,
 'ْ': 44}

meter_class_to_label = {
 0: 'البسيط',
 1: 'الخفيف',
 2: 'الرجز',
 3: 'الرمل',
 4: 'السريع',
 5: 'الطويل',
 6: 'الكامل',
 7: 'المتدارك',
 8: 'المتقارب',
 9: 'المجتث',
 10: 'المديد',
 11: 'المضارع',
 12: 'المقتضب',
 13: 'المنسرح',
 14: 'الهزج',
 15: 'الوافر',
 16: 'نثر'
}

meter_label_to_class = {
    "البسيط": 0,
    "الخفيف": 1,
    "الرجز": 2,
    "الرمل": 3,
    "السريع": 4,
    "الطويل": 5,
    "الكامل": 6,
    "المتدارك": 7,
    "المتقارب": 8,
    "المجتث": 9,
    "المديد": 10,
    "المضارع": 11,
    "المقتضب": 12,
    "المنسرح": 13,
    "الهزج": 14,
    "الوافر": 15,
    "نثر": 16,
}

In [36]:
resamplers = {  # all three sampling rates exist in test split
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    32000: torchaudio.transforms.Resample(32000, 16000),
}

def get_resampled_speech(file_path):
    original_speech, sampling_rate = torchaudio.load(file_path)
    if sampling_rate in resamplers:
        speech = resamplers[sampling_rate](original_speech[0]).squeeze().numpy()
    else:
        speech = resamplers[48000](original_speech[0]).squeeze().numpy()
    return speech

In [37]:
class TextClassifier:
    def __init__(self, transcriber, model_path='/content/drive/MyDrive/KFUPM-Master/ICS606/Models/arabic_poetry_text_classification_model_Gheith.h5'):
        self.model = tf.keras.models.load_model(model_path,)
        self.transcriber = transcriber

    def classify(self, file_path=None,speech_array=None):
        predicted_text = self.transcriber.transcribe(file_path,speech_array)[0]
        predicted_text = araby.strip_tashkeel(predicted_text)
        sequence = [char2idx[char]+1 for char in predicted_text if char in char2idx.keys()]
        sequence = pad_sequences([sequence], maxlen=128, padding="post", value=0)
        predictions = self.model.predict(sequence)[0]
        class_ = np.argmax(predictions, 0).astype("int")
        return meter_class_to_label[class_]

In [38]:
# MODEL_ID = "/content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600"
MODEL_ID = "bakrianoo/sinai-voice-ar-stt"

In [39]:
transcriber = Transcriber(model_path=MODEL_ID)

In [40]:
classifier = TextClassifier(transcriber=transcriber)

In [41]:
# regarding the warnings:
# https://github.com/kensho-technologies/pyctcdecode/blob/main/tutorials/02_pipeline_huggingface.ipynb
# https://huggingface.co/blog/wav2vec2-with-ngram
ctc_transcriper = CTCTranscriber(model_path=MODEL_ID)

In [42]:
classifier_with_ctc_transcibier = TextClassifier(transcriber=ctc_transcriper)

In [21]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [22]:
test_dataset = load_dataset("MagedSaeed/arabic_poetry_speech_benchmark", use_auth_token=True)

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/MagedSaeed___parquet/MagedSaeed--arabic_poetry_speech_benchmark-34fade01103d0c39/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['bait', 'bahr', 'reciter', 'source', 'speech_array'],
        num_rows: 268
    })
})

In [82]:
bahr_en_ar_map = {'baseet':'البسيط',
 'hazaj':'الهزج',
 'kamel':'الكامل',
 'khafeef':'الخفيف',
 'madeed':'المديد',
 'mudhare':'المضارع',
 'mujtath':'المجتث',
 'munsareh':'المنسرح',
 'muqtatheb':'المقتضب',
 'mutadarak':'المتدارك',
 'mutakareb':'المتقارب',
 'rajaz':'الرجز',
 'ramal':'الرمل',
 'saree':'السريع',
 'taweel':'الطويل',
 'wafer':'الوافر',
 }

In [84]:
def predict(example):
  example['predicted'] = classifier.classify(speech_array=example["speech_array"])
  example['ctc_predicted'] = classifier_with_ctc_transcibier.classify(speech_array=example["speech_array"])
  example['bahr_ar'] = bahr_en_ar_map[example['bahr']]
  return example

In [85]:
test_dataset = test_dataset.map(predict)

  0%|          | 0/268 [00:00<?, ?ex/s]

In [88]:
def compute_metrics(preds,labels):
    print('labels:',labels)
    print('predictions:',preds)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds)
    matrix = confusion_matrix(labels, preds)
    print(matrix)
    print('clasification report',report)
    print('accuracy',acc)

In [91]:
compute_metrics(
    preds=[example['predicted'] for example in test_dataset['train']],
    labels=[example['bahr_ar'] for example in test_dataset['train']]
  )

labels: ['الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المد

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [92]:
compute_metrics(
    preds=[example['ctc_predicted'] for example in test_dataset['train']],
    labels=[example['bahr_ar'] for example in test_dataset['train']]
  )

labels: ['الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الهزج', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'الرمل', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المتقارب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'المقتضب', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'الكامل', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المديد', 'المد

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
